In [2]:
import requests
import os

import pandas as pd
from dotenv import load_dotenv
import yfinance as yf

In [3]:
ticker_df = pd.read_csv('data/nasdaq_screener_2025_03_26.csv')

In [4]:
ticker_df.Symbol.sort_values().head()

0        A
1       AA
2    AACBU
3     AACG
4     AACT
Name: Symbol, dtype: object

In [12]:
load_dotenv()

True

In [56]:
res = requests.get(f'https://www.alphavantage.co/query?function=LISTING_STATUS&apikey={os.environ["ALPHAVANTAGE_API_KEY"]}')

In [84]:
with open("data/all_tickers.csv", "wb") as file:
    file.write(res.content)

In [5]:
listings = pd.read_csv('data/all_tickers.csv')
listings.exchange.unique()

array(['NYSE', 'NYSE ARCA', 'BATS', 'NASDAQ', 'NYSE MKT'], dtype=object)

In [7]:
listings.head()

,symbol,name,exchange,assetType,ipoDate,delistingDate,status
0,A,Agilent Technologies Inc,NYSE,Stock,1999-11-18,NaN,Active
1,AA,Alcoa Corp,NYSE,Stock,2016-10-18,NaN,Active
2,AAA,ALTERNATIVE ACCESS FIRST PRIORITY CLO BOND ETF,NYSE ARCA,ETF,2020-09-09,NaN,Active
3,AAAU,Goldman Sachs Physical Gold ETF,BATS,ETF,2018-08-15,NaN,Active
4,AACBU,Artius II Acquisition Inc - Units (1 Ord Shs &...,NASDAQ,Stock,2025-02-13,NaN,Active


In [79]:
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&outputsize=full&apikey={os.environ["ALPHAVANTAGE_API_KEY"]}'
r = requests.get(url)
data = r.json()

data.keys()

dict_keys(['Meta Data', 'Time Series (Daily)'])

In [80]:
pd.DataFrame(data['Time Series (Daily)']).T

,1. open,2. high,3. low,4. close,5. volume
2025-03-26,250.8700,254.3200,249.5300,250.3700,4449369
2025-03-25,248.3600,250.9000,248.2000,249.9000,3133809
2025-03-24,247.3100,248.8200,245.9650,248.4500,4753331
2025-03-21,241.6900,245.2050,238.5000,243.8700,9580084
2025-03-20,244.2400,246.8000,237.2240,243.3200,7026796
...,...,...,...,...,...
1999-11-05,92.7500,92.9400,90.1900,90.2500,13737600
1999-11-04,94.4400,94.4400,90.0000,91.5600,16697600
1999-11-03,95.8700,95.9400,93.5000,94.3700,10369100
1999-11-02,96.7500,96.8100,93.6900,94.8100,11105400


In [14]:
def batch(data: list, batch_size: int):
    for i in range(0, len(data), batch_size):
        yield data[i: i + batch_size]

In [16]:
for chunck in batch(list(listings.symbol), 10):
    print(chunck)

['A', 'AA', 'AAA', 'AAAU', 'AACBU', 'AACG', 'AACT', 'AACT-U', 'AACT-WS', 'AADR']
['AAL', 'AAM', 'AAM-U', 'AAM-WS', 'AAME', 'AAMI', 'AAOI', 'AAON', 'AAP', 'AAPB']
['AAPD', 'AAPG', 'AAPGV', 'AAPL', 'AAPR', 'AAPU', 'AAPW', 'AAPX', 'AAPY', 'AARD']
['AAT', 'AAVM', 'AAXJ', 'AB', 'ABAT', 'ABBV', 'ABCB', 'ABCL', 'ABCS', 'ABEO']
['ABEQ', 'ABEV', 'ABFL', 'ABG', 'ABHY', 'ABL', 'ABLD', 'ABLG', 'ABLLL', 'ABLLW']
['ABLV', 'ABLVW', 'ABM', 'ABNB', 'ABNY', 'ABOS', 'ABOT', 'ABPWW', 'ABR', 'ABR-P-D']
['ABR-P-E', 'ABR-P-F', 'ABSI', 'ABST', 'ABT', 'ABTS', 'ABUS', 'ABVC', 'ABVE', 'ABVEW']
['ABVX', 'AC', 'ACA', 'ACAD', 'ACB', 'ACCD', 'ACCO', 'ACCS', 'ACDC', 'ACEL']
['ACES', 'ACET', 'ACGL', 'ACGLN', 'ACGLO', 'ACGR', 'ACHC', 'ACHL', 'ACHR', 'ACHR-WS']
['ACHV', 'ACI', 'ACIC', 'ACIO', 'ACIU', 'ACIW', 'ACLC', 'ACLO', 'ACLS', 'ACLX']
['ACM', 'ACMR', 'ACN', 'ACNB', 'ACNT', 'ACOG', 'ACON', 'ACONW', 'ACP', 'ACP-P-A']
['ACR', 'ACR-P-C', 'ACR-P-D', 'ACRE', 'ACRS', 'ACRV', 'ACSI', 'ACST', 'ACT', 'ACTG']
['ACTU', 'ACTV',

In [11]:
data['Adj Close'].Q.isna().all()

np.True_

In [26]:
def download_one_ticker(ticker: str) -> pd.DataFrame:
    stock = yf.Ticker(ticker)
    data = stock.history(period='2y', interval='1d')
    return data

In [27]:
download_one_ticker('TSLA')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-03-29 00:00:00-04:00,193.130005,195.289993,189.440002,193.880005,123660000,0.0,0.0
2023-03-30 00:00:00-04:00,195.580002,197.330002,194.419998,195.279999,110252200,0.0,0.0
2023-03-31 00:00:00-04:00,197.529999,207.789993,197.199997,207.460007,170222100,0.0,0.0
2023-04-03 00:00:00-04:00,199.910004,202.690002,192.199997,194.770004,169545900,0.0,0.0
2023-04-04 00:00:00-04:00,197.320007,198.740005,190.320007,192.580002,126463800,0.0,0.0
...,...,...,...,...,...,...,...
2025-03-24 00:00:00-04:00,258.079987,278.640015,256.329987,278.390015,169079900,0.0,0.0
2025-03-25 00:00:00-04:00,283.600006,288.200012,271.279999,288.140015,150361500,0.0,0.0
2025-03-26 00:00:00-04:00,282.660004,284.899994,266.510010,272.059998,156254400,0.0,0.0


In [30]:
tickers = next(batch(list(listings.symbol), 10))
tickers


['A',
 'AA',
 'AAA',
 'AAAU',
 'AACBU',
 'AACG',
 'AACT',
 'AACT-U',
 'AACT-WS',
 'AADR']

In [33]:
import asyncio
import nest_asyncio

tickers = next(batch(list(listings.symbol), 10))

async def download_one_ticker(ticker: str) -> pd.DataFrame:
    try:
        stock = yf.Ticker(ticker)
        data = stock.history(period='2y', interval='1d')

        if data.empty:
            print(f"No data found for ticker {ticker}")
            return None
        
        return data
    except Exception as e:
        print(f" Successfully fetched {ticker}")
        return None

async def download_multiple_tickers():
    tasks = [download_one_ticker(ticker) for ticker in tickers]
    results = await asyncio.gather(*tasks)
    
    valid_data = {tickers[i]: results[i] for i in range(len(tickers)) if results[i] is not None}

    if valid_data:
        combined_df = pd.concat(valid_data.values(), keys=valid_data.keys())
        combined_df.to_csv('data/stock_data.csv')

nest_asyncio.apply()

await download_multiple_tickers()

$AACT-U: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


No data found for ticker AACT-U


$AACT-WS: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


No data found for ticker AACT-WS


In [23]:
async def main():
    await download_multiple_tickers()
asyncio.create_task(main())

<Task pending name='Task-2373' coro=<main() running at C:\Users\beria\AppData\Local\Temp\ipykernel_14912\2055890285.py:1>>